In [18]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM


In [31]:
help(fetch_movielens)

Help on function fetch_movielens in module lightfm.datasets.movielens:

fetch_movielens(data_home=None, indicator_features=True, genre_features=False, min_rating=0.0, download_if_missing=True)
    Fetch the `Movielens 100k dataset <http://grouplens.org/datasets/movielens/100k/>`_.
    
    The dataset contains 100,000 interactions from 1000 users on 1700 movies,
    and is exhaustively described in its
    `README <http://files.grouplens.org/datasets/movielens/ml-100k-README.txt>`_.
    
    Parameters
    ----------
    
    data_home: path, optional
        Path to the directory in which the downloaded data should be placed.
        Defaults to ``~/lightfm_data/``.
    indicator_features: bool, optional
        Use an [n_users, n_users] identity matrix for item features. When True with genre_features,
        indicator and genre features are concatenated into a single feature matrix of shape
        [n_users, n_users + n_genres].
    genre_features: bool, optional
        Use a [n_us

In [19]:
#fetch and format data for minimum rating of 4
data = fetch_movielens(min_rating=4.0) 

In [20]:
print(data)

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>, 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>, 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 1682 stored elements in Compressed Sparse Row format>, 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object), 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}


In [21]:
#print the training and testing set matrices
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [22]:

#setup the model 
model = LightFM(loss="warp")

In [23]:
print(model)

In [24]:
#train the model with the data
model.fit(data["train"], epochs=30, num_threads=2)

In [33]:
#defining a function where for the number of users and movies in training data, 
#generate recommendations for each user we input
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data ["train"].shape
    for user_id in user_ids:
        
        #movies the user likes
        known_positives = data["item_labels"][data['train'].tocsr()[user_id].indices]
        
        #scores for movies these users are predicted to like based on the model
        scores = model.predict(user_id, np.arange(n_items))
        
        #rank the predictions in order of rating
        top_items = data["item_labels"][np.argsort(-scores)]
        
        #print the results
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)
            
sample_recommendation(model, data, [3, 25, 450,101, 67])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        Scream (1996)
        Contact (1997)
        Chasing Amy (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        English Patient, The (1996)
        Fargo (1996)
        Contact (1997)
User 450
     Known positives:
        Contact (1997)
        George of the Jungle (1997)
        Event Horizon (1997)
     Recommended:
        Conspiracy Theory (1997)
        Saint, The (1997)
        Air Force One (1997)
User 101
     Known positives:
        Star Wars (1977)
        Blade Runner (1982)
        Aladdin (1992)
     Recommended:
        Raiders of the Lost Ark (1981)
        Star Wars (1977)
        Empire Strikes Back, The (1980)
User 67
     Known positives:
        Dead Man Walking (1995)
        Birdcage, The (1996)
        Star Wars (1977)
     Recommended:
  